### Why CNNs?

the fundamental of this problem deals with surrounding blocks. Kernels in CNNs are better suited for this task as they can sense the changes in surrounding blocks. => less filters with added sense of sequences.

### Adding a sense of Seqence to the Network

Maybe combine CNN+LSTM or something new

Something like CNN+Sequences or Auto-regressive CNNs with final target

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers as L

from matplotlib import pyplot as plt

In [ ]:
train_df = pd.read_csv("/kaggle/input/conways-reverse-game-of-life-2020/train.csv")
test_df = pd.read_csv("/kaggle/input/conways-reverse-game-of-life-2020/test.csv")

sample_submission = pd.read_csv("/kaggle/input/conways-reverse-game-of-life-2020/sample_submission.csv")

In [ ]:
train_df.head(2)

In [ ]:
start_features = [f for f in train_df.columns if "start" in f]
stop_features = [f for f in train_df.columns if "stop" in f]

features_in = stop_features+["delta"]

### Let's plot a sample

In [ ]:
idx = 3

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle(f'Delta: {train_df.loc[idx, "delta"]}')
ax1.imshow(1-(train_df.loc[idx, start_features].values).reshape(25, 25), cmap="gray")
ax1.set_title("Start Setting")
ax2.imshow(1-(train_df.loc[idx, stop_features].values).reshape(25, 25), cmap="gray")
ax2.set_title("Stop Setting")

#### Keras dataset Sequence

In [ ]:
@tf.function
def _features_to_img(features):
    
    return tf.cast(features, tf.float32)

class LifeSet(tf.keras.utils.Sequence):

    def __init__(self, df:pd.DataFrame, is_train:bool, batch_size:int=32):
        self.df = df
        self.is_train=is_train
        self.batch_size = batch_size
        
        if is_train:
            self.stop = df.loc[:, stop_features].values.reshape(-1, 25, 25, 1)
            self.start = df.loc[:, start_features].values.reshape(-1, 25, 25, 1)
            self.delta = df.loc[:, "delta"]
            
        else:
            self.stop = df.loc[:, stop_features].values.reshape(-1, 25, 25, 1)
            self.delta = df.loc[:, "delta"]

    def __len__(self):
        return len(self.df)
    

    def __getitem__(self, idx):
        
        print(idx)
        
        if self.is_train:
            
            batch_x = self.stop[idx*self.batch_size:(idx+1)*self.batch_size]
            deltas = self.delta[idx*self.batch_size:(idx+1)*self.batch_size]
            batch_y = self.start[idx*self.batch_size:(idx+1)*self.batch_size]

            return [deltas, _features_to_img(batch_x)], _features_to_img(batch_y)
        
        else:
            
            batch_x = self.stop[idx*self.batch_size:(idx+1)*self.batch_size]
            deltas = self.delta[idx*self.batch_size:(idx+1)*self.batch_size]
            
            return {
                "delta":deltas,
                "stop":_features_to_img(batch_x)
            }

In [ ]:
bs=1 #Setting Batch size to 1 as the model involves a loop over delta
train_set = LifeSet(train_df, is_train=True, batch_size=bs)
test_set = LifeSet(test_df, is_train=False, batch_size=bs)

In [ ]:
tr_data = next(iter(train_set))
tst_data = next(iter(test_set))

In [ ]:
tr_df = tf.data.Dataset.from_tensor_slices((
    (train_df["delta"].values, train_df[stop_features].values.reshape(-1, 25, 25, 1).astype(float)), 
    train_df[start_features].values.reshape(-1, 25, 25, 1).astype(float)))

tr_df = tr_df.batch(128)



In [ ]:
tst_df = tf.data.Dataset.from_tensor_slices((
    (test_df["delta"].values, test_df[stop_features].values.reshape(-1, 25, 25, 1).astype(float)), ))

tst_df = tst_df.batch(128)

In [ ]:
tst_df

In [ ]:
class LifeModel(tf.keras.Model):
    
    def __init__(self):
        super(LifeModel, self).__init__()
        
        
        self.encoder = L.Conv2D(32, kernel_size=(3,3), padding="SAME")
        self.memory = tf.keras.models.Sequential([
            L.Conv2D(128, kernel_size=(3,3), padding="SAME"),
            L.BatchNormalization(),
            L.ReLU(),
            
            L.Conv2D(32, kernel_size=(3,3), padding="SAME"),
            L.BatchNormalization(),
            L.ReLU(),
        ])
        self.decoder = L.Conv2D(1, kernel_size=(3,3), padding="SAME")
        
    def call(self, inputs):
        
        #print(inputs)
        
        delta = inputs[0]
        stop_state = inputs[1]
        
        print(tf.reshape(delta, (-1,1)))
        
        x = self.encoder(stop_state-0.5)
        x = tf.nn.relu(x)
        
        print(x.shape)
        
        #TODO: make this custom for each delta in a batch. 
        #Maybe a custom train_step in keras  model
        for i in range(tf.reduce_max(delta)):
            x += self.memory(x)
        
        x = self.decoder(x)
        #x = tf.nn.relu(x)
        x = tf.math.sigmoid(x)
        
        return x

In [ ]:
model = LifeModel()
model.compile(loss="bce", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

#ps = model(tr_data[0])
#ps.shape

In [ ]:
model.fit(tr_df, epochs=50)

### Let's take some samples

In [ ]:
tr_data = next(iter(tr_df))

In [ ]:
ps = model(tr_data[0])

In [ ]:
ps.shape

In [ ]:
idx = 5

fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
fig.suptitle("Delta: "+str(tr_data[0][0][idx]))

ax1.imshow(1-(tr_data[0][1][idx].numpy().reshape(25, 25)), cmap="gray")
ax1.set_title("Stop Setting")

ax2.imshow(1-(tr_data[1][idx].numpy().reshape(25, 25)), cmap="gray")
ax2.set_title("Start Setting")

ax3.imshow(1-(ps[idx]>=0.5).numpy().reshape(25, 25), cmap="gray")
ax3.set_title("Predicted Setting")

### Generating output

In [ ]:
ps = model.predict(tst_df, verbose=1)
plt.imshow(1-(ps[0]>=0.5).reshape(25, 25), cmap="gray")

In [ ]:
test_df

In [ ]:
THRESH = 0.5

ps_=(ps>=THRESH).astype(int).reshape(test_df.shape[0], -1)

In [ ]:
ps_.shape

In [ ]:
sample_submission

In [ ]:
sub = test_df[["id"]].copy()
tmp = pd.DataFrame(ps_, columns=start_features)
sub = sub.join(tmp)

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)